In [2]:
from datatrove.pipeline.filters import (
    GopherQualityFilter,
    GopherRepetitionFilter)

In [ ]:
GopherRepetitionFilter()

In [8]:
import sys
sys.path.insert(0,'..')

In [9]:
from utils import *

In [11]:
! ls ./Sample

1000.jsonl


In [12]:
data = load_jsonl('./Sample/1000.jsonl')

1000it [00:00, 30147.52it/s]


In [14]:
import re

In [22]:
from datatrove.pipeline.writers.disk_base import DiskWriter
from datatrove.pipeline.filters.gopher_repetition_filter import *

In [53]:
okt = Okt()

In [55]:
okt.morphs("열심히 코딩한 당신, 연휴에는 여행을 안갔어")

['열심히', '코딩', '한', '당신', ',', '연휴', '에는', '여행', '을', '안', '갔어']

In [86]:
class OKGopherRepetitionFilter(GopherRepetitionFilter):
    def __init__(
        self,
        dup_line_frac: float | None = 0.3,
        dup_para_frac: float | None = 0.3,
        dup_line_char_frac: float | None = 0.2,
        dup_para_char_frac: float | None = 0.2,
        top_n_grams: tuple[tuple[int, float]] = ((2, 0.2), (3, 0.18), (4, 0.16)),
        dup_n_grams: tuple[tuple[int, float]] = ((5, 0.15), (6, 0.14), (7, 0.13), (8, 0.12), (9, 0.11), (10, 0.10)),
        exclusion_writer: DiskWriter = None,
    ):
        super().__init__(dup_line_frac, dup_para_frac, dup_line_char_frac, dup_para_char_frac, top_n_grams, dup_n_grams, exclusion_writer)
        self._line_splitter = re.compile("\n+")

    def filter(self, doc: str) -> bool | tuple[bool, str]:
        from konlpy.tag import Okt
        okt = Okt()
        text = doc#.text

        paragraphs = self.paragraph_exp.split(text.strip())
        paragraphs_duplicates, char_duplicates = find_duplicates(paragraphs)
        if self.dup_para_frac and paragraphs_duplicates / len(paragraphs) > self.dup_para_frac:
            return False, "dup_para_frac"
        if self.dup_para_char_frac and char_duplicates / len(text) > self.dup_para_char_frac:
            return False, "dup_para_char_frac"

        lines = self._line_splitter.split(text)
        line_duplicates, char_duplicates = find_duplicates(lines)
        if self.dup_line_frac and line_duplicates / len(lines) > self.dup_line_frac:
            return False, "dup_line_frac"
        if self.dup_line_char_frac and char_duplicates / len(text) > self.dup_line_char_frac:
            return False, "dup_line_char_frac"

        words = okt.morphs(text)  # TODO we should use language id filter

        for n, n_frac in self.top_n_grams:
            n_grams = get_n_grams(words, n)
            if not n_grams:
                continue
            top_char_length = find_top_duplicate(n_grams) # n_gram의 char 개수 * freq
            if top_char_length / len(text) > n_frac:
                return False, f"top_{n}_gram"

        for n, n_frac in self.dup_n_grams:
            n_duplicates_char = find_all_duplicate(words, n)
            if n_duplicates_char / len(text) > n_frac:
                return False, f"duplicated_{n}_n_grams"

        return True

In [85]:
tmp = okt.morphs(data[0]['raw_content'])

In [63]:
def get_n_grams(words: list[str], n: int) -> list[str]:
    return [" ".join(words[i : i + n]) for i in range(len(words) - n + 1)]


In [84]:
def find_all_duplicate(words: list[str], n: int) -> int:
    n_words = len(words)
    unique = set()
    repeated_chars, idx = 0, 0
    while idx < n_words - n + 1:
        n_gram = "".join(words[idx : idx + n])
        if n_gram in unique:
            repeated_chars += len(n_gram)
            idx += n
        else:
            unique.add(n_gram)
            idx += 1
    assert repeated_chars <= len("".join(words))
    return repeated_chars

In [83]:
def find_top_duplicate(x: list[str]) -> int:
    counter = Counter()
    for element in x:
        counter[element] += 1
    top_n_gram = counter.most_common(1)[0]
    return len(top_n_gram[0]) * top_n_gram[1]

In [87]:
gopher_filter = OKGopherRepetitionFilter()

In [108]:
check = []
for i in tqdm(data):
    check.append(gopher_filter.filter(i['raw_content']))

100% 1000/1000 [01:07<00:00, 14.74it/s]


In [91]:
from collections import Counter

In [107]:
332/1000

0.332

In [106]:
1000-668

332

In [92]:
Counter(check)

Counter({True: 668,
         (False, 'top_2_gram'): 96,
         (False, 'top_4_gram'): 77,
         (False, 'top_3_gram'): 74,
         (False, 'duplicated_5_n_grams'): 72,
         (False, 'duplicated_7_n_grams'): 4,
         (False, 'duplicated_8_n_grams'): 3,
         (False, 'duplicated_9_n_grams'): 3,
         (False, 'duplicated_10_n_grams'): 2,
         (False, 'duplicated_6_n_grams'): 1})

In [94]:
for i,j in zip(data, check):
    if j!=True:
        print(j)
        print(i['raw_content'])
        input()

(False, 'duplicated_8_n_grams')
추락 방지 키트
Miller 루핑 추락 방지 키트
Miller 루핑 추락 방지 키트는 가파른 지붕이나 평평한 지붕에 쉽게 설치하고 사용할 수 있습니다.
특징 및 이점:Miller 지붕 추락 방지 키트는 가파른 지붕이나 평평한 지붕에 쉽게 설치하고 사용할 수 있습니다.
재사용 가능한 지붕 앵커 2개
나일론 로프 구명줄
수동 스테인리스 스틸 로프 그랩과 2ft.(6m)웹 랜야드와 충격 흡수 장치
DuraFlex 신축성 하네스(E850/UGN)
하니스 크기
로프 구명줄 재질
하니스 스타일
하니스 크기 : 범용
로프 구명줄 재질 : 나일론
하니스 스타일 : 전신
Temporary Roof Anchor & System Kit user Manuals


(False, 'top_3_gram')
[일반] 3060ti 견적 이렇게 구매.. [2]


(False, 'top_4_gram')
도서필립스 신약 성경 세트
J. B. 필립스 저/김명희,송동민 공역 아바서원 | 2020년 08월


(False, 'duplicated_5_n_grams')
filter #그누보드5 ×
단축 URL 사용 시 페이지 이동하면 오류가 나요 ㅜ #그누보드5
밀리지 자기소개 전체게시물 회원게시물 회원 질문검색 회원 답변검색 회원 댓글검색
특정 게시판 검색 기능 #검색 #그누보드5
seoy 자기소개 전체게시물 회원게시물 회원 질문검색 회원 답변검색 회원 댓글검색
그누보드로 개설한 사이트 제거 #그누보드5
김두웅 자기소개 전체게시물 회원게시물 회원 질문검색 회원 답변검색 회원 댓글검색
인스타 데이터 가져올수 있나요 #그누보드5
빠왕 홈페이지 자기소개 전체게시물 회원게시물 회원 질문검색 회원 답변검색 회원 댓글검색
토글 스위치 링크이동? #그누보드5
그누초초초초보 자기소개 전체게시물 회원게시물 회원 질문검색 회원 답변검색 회원 댓글검색
가로스크롤 #그누보드5 #팝업 #스크롤
웹호스팅 후 파일 내려받고 xampp로 수정하는법 알려주실 수 있을까요? #그누보드5


(False, 'top_2_gram')
https://kr.freepik.com/premium-vector/concept-of-online-tool-or-service-for-social-media-analytics-and-smm-web-banner-or-landing-page_29268695.htm
Avifors 506개의 리소스
소셜 미디어 분석 및 Smm 웹 배너 또는 방문 페이지를 위한 온라인 도구 또는 서비스의 개념


(False, 'duplicated_5_n_grams')
도서(자료별) (2023. 1. 1. 현재)
안동도서관 자료별 자료현황
향토도서
권 140,099 71,099 1,013 621 580 213,412
도서(주제별) (2023. 1. 1. 현재)
안동도서관 주제별 자료현황
권 5,623 11,918 5,722 32,087 12,792 15,033 8,323 7,874 96,438 17,602 213,412
비도서 및 전자자료 (2023. 1. 1. 현재)
비도서 및 전자자료
점 11,099 2,257 94 36 13,486
연속간행물자료 (2023. 1. 1. 현재)
연속간행물자료 자료현황
기증
종 32 14 20 79 171 316


(False, 'duplicated_10_n_grams')
보웬스 호환 어댑터를 팔각 소프트박스에 사용가능할까요
5 푸른안내자 사진 조명 4 1514 0 0 2019.07.11 21:44
덥고 습하고 비오고 완전 여름이네요.
건강 유의하세요.
이 제품인데요.
고독스 팔각 소프트박스, 우산형이구요.
우산 중심대를 어댑터에 끼워야 하는데
보웬스 s-type 어댑터의 우산 꽂는 구멍에 꽂으면 스피드라이트 뒷부분을 접는게 아니라 펴야 되는 것 같아요.
그럼 이 소프트박스 안에 들어가 있을까요?
일반 엄브렐라가 아니라 디퓨저 천도 달아야 하는 소프트박스입니다.
아무리 검색해도 안 나와서 이렇게 질문글 올려봅니당.
Lv.5 5 푸른안내자 골드
5 푸른안내자 2019.07.11 21:47
사진첨부가 안 돼서 댓글에 남깁니다. 이 팔각 소프트박스입니다.
이런 엄브렐러 어댑터만 가능한 소프트박스일까요.
[https://strobistkorea.com/data/editor/1907/img_3077355859_GPDH4R9Z_9553e0698a56905c7d67b7314fb55a449b2de5fc.jpg] 사진첨부가 안 돼서 댓글에 남깁니다. 이 팔각 소프트박스입니다. [https://strobistkorea.com/data/editor/1907/img_3077355859_NfTxqyuO_3e83d8fc4539a65509576f93dbf585230cc380cb.jpg] 이런 엄브렐러 어댑터만 가능한 소프트박스일까요.
S-Type브라켓 안에 넣어서도 접을 수 있는데요...스피드라이트 해드부분을 넣어서 체결하고 나머지 부분을 접으면 됩니다..
위 소프트 박스 사용하는데 전혀 지장없어요..
우산 넣는 곳에 걸리면 반대방향으로 넣어서 접으면 ....그리고 딱히 안 접어도 공간은 충분하구요 ,,,,
S-Type브라켓 안에 넣어서도 접을 수 있는데요...스피드라이트 해드부분을 넣어서 체결하고 나머지 부분을 접으면 됩니다.. 위 소프트 박스 사용하는데 전혀 지장없어요.. 우산 넣는 곳에 걸리

(False, 'duplicated_5_n_grams')
항공기가스터빈엔진정비
항공기가스터빈엔진정비는 항공기가스터빈엔진의 성능을 유지하기 위하여 부품의 상태점검, 분해조립, 세척, 수리, 교환, 검사, 작동점검 및 시험업무를 하는 일이다.
항공기가스터빈엔진 사전점검(구버전) 수준 : 3
흡입계통 검사하기
유압계통 검사하기
엔진마운트 검사하기
압축계통 검사하기
연소계통 검사하기
터빈계통 검사하기
배기계통 검사하기
기어박스 검사하기
연료계통 검사하기
오일계통 검사하기
공기압계통 검사하기
항공기가스터빈엔진 외부 장착품 장·탈착 수준 : 3
항공기가스터빈엔진 외부장착품 장·탈착이란 가스터빈엔진이 모듈단위로 분리되지않은 상태에서 엔진의 외부에 장착된 전기배선, 보기부품, 엔진마운트를 장·탈착하는능력이다.
전기배선 장·탈착하기
보기부품 장·탈착하기
엔진마운트 장·탈착하기
공기압계통
스러스트
엔진마운팅
연료계통
전기배선
탈착작업
항공관련학과
항공기가스터빈엔진 모듈 장·탈착 수준 : 4
항공기가스터빈엔진 모듈 장·탈착이란 엔진을 모듈 단위로 분리하여 장탈한 다음 다시완성된 엔진으로 결합시키는 능력이다.
압축기모듈 장·탈착하기
연소실모듈 장·탈착하기
터빈모듈 장·탈착하기
배기장치 장·탈착하기
기어박스모듈 장·탈착하기
연소실
이동장비
특수장비
항공기가스터빈엔진 모듈 분해조립 수준 : 4
항공기가스터빈엔진 모듈 분해조립이란 가스터빈엔진을 구성하는 각 모듈을 점검하고수리하기 위해서 압축기, 연소실, 터빈, 배기장치, 기어박스 모듈을 분해하고 조립하는 능력이다.
압축기모듈 분해조립하기
연소실모듈 분해조립하기
터빈모듈 분해조립하기
배기장치 분해조립하기
기어박스모듈 분해조립하기
터빈
항공기가스터빈엔진 부품세척 수준 : 2
항공기가스터빈엔진 부품세척이란 엔진부품의 품질을 보증하고, 신뢰성 있는 검사와수리를 위하여 일반 세척, 기계 세척, 약품 세척하는 능력이다.
일반 세척하기
기계 세척하기
약품 세척하기
블라스트
세척장비
약품세척
연마재
항공엔진 기능사
항공기가스터빈엔진 부품검사 수준 :

KeyboardInterrupt: Interrupted by user

In [95]:
for i,j in zip(data, check):
    i['filtered']=j

In [98]:
j

True

In [100]:
subset = {}

In [102]:
for i in data:
    if i['filtered']!=True:
        if subset.get(i['filtered']):
            continue
        else:
            subset[i['filtered']]=i

In [105]:
for i,j in subset.items():
    print(i)
    print(j['raw_content'])
    print()

(False, 'duplicated_8_n_grams')
추락 방지 키트
Miller 루핑 추락 방지 키트
Miller 루핑 추락 방지 키트는 가파른 지붕이나 평평한 지붕에 쉽게 설치하고 사용할 수 있습니다.
특징 및 이점:Miller 지붕 추락 방지 키트는 가파른 지붕이나 평평한 지붕에 쉽게 설치하고 사용할 수 있습니다.
재사용 가능한 지붕 앵커 2개
나일론 로프 구명줄
수동 스테인리스 스틸 로프 그랩과 2ft.(6m)웹 랜야드와 충격 흡수 장치
DuraFlex 신축성 하네스(E850/UGN)
하니스 크기
로프 구명줄 재질
하니스 스타일
하니스 크기 : 범용
로프 구명줄 재질 : 나일론
하니스 스타일 : 전신
Temporary Roof Anchor & System Kit user Manuals

(False, 'top_3_gram')
[일반] 3060ti 견적 이렇게 구매.. [2]

(False, 'top_4_gram')
도서필립스 신약 성경 세트
J. B. 필립스 저/김명희,송동민 공역 아바서원 | 2020년 08월

(False, 'duplicated_5_n_grams')
filter #그누보드5 ×
단축 URL 사용 시 페이지 이동하면 오류가 나요 ㅜ #그누보드5
밀리지 자기소개 전체게시물 회원게시물 회원 질문검색 회원 답변검색 회원 댓글검색
특정 게시판 검색 기능 #검색 #그누보드5
seoy 자기소개 전체게시물 회원게시물 회원 질문검색 회원 답변검색 회원 댓글검색
그누보드로 개설한 사이트 제거 #그누보드5
김두웅 자기소개 전체게시물 회원게시물 회원 질문검색 회원 답변검색 회원 댓글검색
인스타 데이터 가져올수 있나요 #그누보드5
빠왕 홈페이지 자기소개 전체게시물 회원게시물 회원 질문검색 회원 답변검색 회원 댓글검색
토글 스위치 링크이동? #그누보드5
그누초초초초보 자기소개 전체게시물 회원게시물 회원 질문검색 회원 답변검색 회원 댓글검색
가로스크롤 #그누보드5 #팝업 #스크롤
웹호스팅 후 파일 내려받고 xampp로 수정하는법 알려주실 수 있을까요? #그